In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, InputLayer
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam



import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import os 

In [2]:
def set_globals():
    global open3D_directory
    global dataset_name_num
    global MIF_nodes 
    global xyz_offset
    open3D_directory = "C:\open3dtools"
    dataset_name_num = "06.DHFR"
    MIF_nodes = [23,27,21]
    xyz_offset = [-11,-10,-9]
set_globals()

In [3]:
def import_data():
    path = os.path.join(r"C:\Users\Linden\GitHub\3DQSAR\data\LabelledData",dataset_name_num)
    labels = pd.read_csv(os.path.join(path,"pIC50.CSV"))
    Y = labels["Y"].to_numpy()

    data=[]

    for entry in os.scandir(path):
        if (entry.path.endswith(".npy")):
            file = np.load(entry.path)
            data.append(file)
    field_1 = data[0::2]
    field_2 = data[1::2]
    X_1 = np.stack(field_1,0)
    X_2 = np.stack(field_2,0)    
    
    return X_1, X_2, Y

In [4]:
X_1, X_2, Y = import_data()

In [5]:
###Pickle data

In [6]:
###Normalise image data 
###Normalise by amax value?

In [38]:
###Shuffle
permutation = np.random.RandomState(seed=42)\
                .permutation(X_1.shape[0])
X_1 = X_1[permutation]
X_2 = X_2[permutation]
Y = Y[permutation]

In [39]:
X_1[0].shape

(25, 24, 26)

In [40]:
one=np.arange(306)
two=np.arange(306)[::-1]

In [41]:
st=np.stack([one,two])

In [42]:
st[permutation]

IndexError: index 182 is out of bounds for axis 0 with size 2

In [36]:
permutation

array([182, 154, 111, 203,  60,   9, 119, 157, 167,  33,   5, 101,  45,
       177, 118,  46, 125, 195, 251, 143, 232, 272, 248,  25, 147, 224,
         3,  73, 285, 109, 140, 287, 211,  42,  17, 282,  76,  90,  24,
        57,  92,  77, 254, 116,   7, 193, 284,  78, 295, 237, 223, 291,
        63,  82, 192, 238, 221, 104, 303, 196, 230, 271, 213, 244,  75,
       179,  59,  93,   6, 180,  30,  22, 260,  56, 247, 114, 132, 240,
       204, 275, 165, 220,  84,  66, 113, 168, 146,  19, 144,  79, 255,
       124,  72,  15,  10, 164, 249,  97,  68, 233,  37,  16, 126, 152,
       289,  67, 108,  69,  31, 181, 155, 234, 298,  18, 301,  96, 137,
       292,  86, 258, 172, 209, 259, 294, 139, 242,  55, 173, 290, 194,
       268, 129,  38, 202, 175, 186, 112, 256, 117, 283, 277, 288, 183,
       185,   2, 115, 148, 184, 120, 219, 265, 127,  74,  29,  83, 253,
       107, 158, 163, 133, 197, 225,  65, 206,  85, 222, 159,  12,  35,
        28, 170, 142, 198, 131, 176,  51,  95, 231, 178, 229,  4

In [17]:
z=np.stack([X_1,X_2])

In [8]:
####Train test split
train_upper = int(np.around((X_1.shape[0]*0.9),0))

X_train = X[:train_upper]
X_test = X[train_upper:]

Y_train = Y[:train_upper]
Y_test = Y[train_upper:]

NameError: name 'fake_X_data' is not defined

In [ ]:
###Train validation split
train_upper = int(np.around((X_train.shape[0]*0.8),0))
X_train = X_train[:train_upper]
X_val = X_train[train_upper:]

Y_train = Y_train[:train_upper]
Y_val = Y_train[train_upper:]

In [ ]:
X_train.shape

In [ ]:
train = (X_train, Y_train)
validation = (X_val, Y_val)
test = (X_test, Y_test)

In [ ]:
def build_CNN():
    model = Sequential(name='CNN')
    
    # elu=Exponential Linear Unit, similar to leaky Relu
    #perhaps normalise in layer 1 
    
    # Convolution Layers
    model.add(InputLayer(input_shape=(X_train.shape[1], X_train.shape[2],X_train.shape[3])))
    model.add(MaxPooling2D(2))
    model.add(Conv2D(32, (2, 2), strides=(1, 1), activation='elu'))
    model.add(MaxPooling2D(2))
    model.add(Conv2D(64, (2, 2), strides=(1, 1), activation='elu'))
    model.add(MaxPooling2D(2))
    model.add(Conv2D(128, (2, 2), strides=(1, 1), activation='elu'))
    
    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dropout(0.1))
    #change to sigmoid if pic50 between 0 and 1
    model.add(Dense(2,activation = 'linear'))
    
     
    optimizer = Adam(learning_rate=1e-3)
    model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
    
    return model

model = build_CNN()
print(model.summary())

In [ ]:
##Train 
##monitor early stopping using val_acc
#Set min_delta to 1e-3
monitor = EarlyStopping(monitor='val_loss',
                        min_delta=0,
                        patience=15,
                        verbose=1, 
                        mode='auto',
                        restore_best_weights=True)
model.fit(X_train,
          Y_train,
          batch_size=64,
          epochs=200, 
          validation_data=(X_val, Y_val),
          callbacks=[monitor])


In [ ]:
##Test model w/ best weights on validation data
model_loss = model.evaluate(X_test,y_test_s)
print(f"Model loss:{model_loss}")
